In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.data import sampler
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import random_split
import copy

In [2]:
batch_size = 64
USE_GPU = torch.cuda.is_available()
if USE_GPU:
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print('using device:', device)

# 数据归一化
cifar10_mean = (0.49,0.48,0.45)
cifar10_std = (0.25,0.24,0.26)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(cifar10_mean,cifar10_std)
])

# 训练集
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

# 划分训练集和验证集
NUM_TRAIN = int(0.9*len(trainset)) # 训练集大小，剩余数据用作验证

print('训练集：%d，测试集：%d' % (NUM_TRAIN,len(trainset)-NUM_TRAIN))

trainloader = DataLoader(trainset, batch_size=batch_size,sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))
validateloader = DataLoader(trainset, batch_size=batch_size, sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, len(trainset))))

# 测试集
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

using device: cpu
Files already downloaded and verified
训练集：45000，测试集：5000
Files already downloaded and verified


In [3]:
# 验证模型在验证集或者测试集上的准确率
def check_accuracy(loader, model):
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')
    num_correct = 0
    num_samples = 0
    model.eval()   # set model to evaluation mode
    with torch.no_grad():
        for input, label in loader:
            input = input.to(device=device, dtype=torch.float32)
            label = label.to(device=device, dtype=torch.long)
            scores = model(input)
            _,pred = scores.max(1)
            num_correct += (pred==label).sum()
            num_samples += pred.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f%%)' % (num_correct, num_samples, 100 * acc ))
        return acc

def train_model(model, optimizer, criterion, epochs, scheduler=None):
    best_model_wts = None
    best_acc = 0.0
    model = model.to(device=device) # move the model parameters to CPU/GPU
    for e in range(epochs):
        if scheduler:
            scheduler.step()
        for t,(input, label) in enumerate(trainloader):
            model.train()   # set model to training mode
            input = input.to(device, dtype=torch.float32)
            label = label.to(device, dtype=torch.long)

            outputs = model(input)
            loss = criterion(outputs, label)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print('Epoch %d, loss=%.4f' % (e, loss.item()))
        acc = check_accuracy(validateloader, model)
        if acc > best_acc:
            best_model_wts = copy.deepcopy(model.state_dict())
            best_acc = acc
    print('best_acc:%.2f%%' % (best_acc * 100))
    model.load_state_dict(best_model_wts)
    return model

In [4]:
class MyModel(nn.Module):
    def __init__(self) -> None:
        super(MyModel, self).__init__()
        self.conv1 = nn.Conv2d(3,16,kernel_size=3,padding=1)
        self.batchnorm1 = nn.BatchNorm2d(16)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(2,stride=2)

        self.conv2 = nn.Conv2d(16,32,kernel_size=3,padding=1)
        self.batchnorm2 = nn.BatchNorm2d(32)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(2,stride=2)

        self.conv3 = nn.Conv2d(32,32,kernel_size=3,padding=1)
        self.batchnorm3 = nn.BatchNorm2d(32)
        self.relu3 = nn.ReLU()
        self.maxpool3 = nn.MaxPool2d(2,stride=2)

        self.flatten = nn.Flatten()

        self.liner1 = nn.Linear(32*4*4,128)
        self.liner2 = nn.Linear(128,10)

    def forward(self, x):
        out = self.conv1(x)
        out = self.batchnorm1(out)
        out = self.relu1(out)
        out = self.maxpool1(out)

        out = self.conv2(out)
        out = self.batchnorm2(out)
        out = self.relu2(out)
        out = self.maxpool2(out)

        out = self.conv3(out)
        out = self.batchnorm3(out)
        out = self.relu3(out)
        out = self.maxpool3(out)

        out = self.flatten(out)
        out = self.liner1(out)
        out = self.liner2(out)

        return out

In [5]:
lr = 0.01
momentum = 0.9
epochs = 10
criterion = F.cross_entropy

model = MyModel()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, nesterov=True)
model = train_model(model, optimizer, criterion, epochs)

Epoch 0, loss=0.8116
Checking accuracy on validation set
Got 3041 / 5000 correct (60.82%)
Epoch 1, loss=1.0168
Checking accuracy on validation set
Got 3325 / 5000 correct (66.50%)
Epoch 2, loss=0.6727
Checking accuracy on validation set
Got 3298 / 5000 correct (65.96%)
Epoch 3, loss=1.2998
Checking accuracy on validation set
Got 3538 / 5000 correct (70.76%)
Epoch 4, loss=1.1451
Checking accuracy on validation set
Got 3429 / 5000 correct (68.58%)
Epoch 5, loss=0.8719
Checking accuracy on validation set
Got 3600 / 5000 correct (72.00%)
Epoch 6, loss=0.7022
Checking accuracy on validation set
Got 3722 / 5000 correct (74.44%)
Epoch 7, loss=1.4002
Checking accuracy on validation set
Got 3674 / 5000 correct (73.48%)
Epoch 8, loss=0.6700
Checking accuracy on validation set
Got 3614 / 5000 correct (72.28%)
Epoch 9, loss=1.2593
Checking accuracy on validation set
Got 3673 / 5000 correct (73.46%)
best_acc:74.44%


In [6]:
check_accuracy(testloader, model)

Checking accuracy on test set
Got 7264 / 10000 correct (72.64%)


0.7264

In [7]:
#保存模型
PATH = './cifar_net.pth'
torch.save(model.state_dict(), PATH)